### 세상에 없는 얼굴 GAN 오토인코더 

In [10]:
# 배치 정규화 , 이미지 확대 ,DCGAN

import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import (Activation, BatchNormalization, Conv2D,
                                     Dense, Dropout, Flatten, Input, LeakyReLU,
                                     Reshape, UpSampling2D)
from tensorflow.keras.models import Sequential

df=pd.DataFrame(np.random.randn(100,28,28,1))

NameError: name 'pd' is not defined

In [ ]:
# (x_train,y_train), (x_test,y_test) = mnist.load_data()

In [4]:
# 생성자 모델 생성
generator=Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2))) # 입력층 ->layer 1
generator.add(BatchNormalization()) # 배치 정규화 : 평균이 0, 분산이 1인 데이터로 재배치
generator.add(Reshape((7, 7, 128))) # 3차원으로 데이터 타입 변경
generator.add(UpSampling2D(size=(2, 2))) # 이미지를 2배로 확장시킴 14*14 128
generator.add(Conv2D(64, kernel_size=5, padding='same'))
generator.add(BatchNormalization())
generator.add(UpSampling2D())
generator.add(Conv2D(1, kernel_size=5, padding='same',activation='tanh'))
generator.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 6272)              633472    
                                                                 
 batch_normalization_4 (Batc  (None, 6272)             25088     
 hNormalization)                                                 
                                                                 
 reshape_2 (Reshape)         (None, 7, 7, 128)         0         
                                                                 
 up_sampling2d_4 (UpSampling  (None, 14, 14, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        204864    
                                                                 
 batch_normalization_5 (Batc  (None, 14, 14, 64)      

In [9]:
# 반별자 모델 생성
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5,strides=2,input_shape=(28,28,1), padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5,strides=2, padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1,activation='sigmoid'))
discriminator.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 14, 14, 64)        1664      
                                                                 
 activation (Activation)     (None, 14, 14, 64)        0         
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 7, 7, 128)         204928    
                                                                 
 activation_1 (Activation)   (None, 7, 7, 128)         0         
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0         
                                                                 
 flatten (Flatten)           (None, 6272)             

In [ ]:

discriminator.compile(loss='binary_crossentropy', optimizer=Adam)